In [102]:
import tabula
from tabula import read_pdf 
from tabulate import tabulate 
import pandas as pd
import os
import numpy as np
from datetime import datetime

## Downloading MPD Staffing Data

In [3]:
from urllib.request import urlretrieve

import urllib.request

opener = urllib.request.URLopener()
opener.addheader('User-Agent', 'whatever')

years = [2024,2023,2022,2021,2020,2019,2018]

#2022: Staffing_Report_September%202022.pdf
#2021: Staffing_Report_September%202021.pdf
#2020: Staffing%20Report_July%202020.pdf
#2019: Staffing%20Report_June%202019.pdf
#2018: Staffing%20Report_Table_June%202018.pdf

failed_files = []

for fy_year in years:

    if fy_year in [2024,2023]:
        month_list = ['SEP','AUG','JUL','JUN','MAY','APR','MAR','FEB','JAN','DEC','NOV','OCT']
    else:
        month_list = ['September', 'August', 'July', 'June', 'May', 'April', 'March', 'February', 'January', 'December', 'November','October']

    for month in month_list:

        file_downloaded = 0

        if month in ['DEC','NOV','OCT','December', 'November', 'October']:
            year = fy_year - 1 
        else:
            year = fy_year

        try:
            if fy_year in [2024,2023]:
                if (month == 'MAR') & (year == 2023):
                    link = "https://mpdc.dc.gov/sites/default/files/dc/sites/mpdc/page_content/attachments/Staffing_Report_{}_{}_updated.pdf".format(month,year)
                else:
                    link = "https://mpdc.dc.gov/sites/default/files/dc/sites/mpdc/page_content/attachments/Staffing_Report_{}_{}.pdf".format(month,year)
                filename, headers = opener.retrieve(link, 'Staffing_Report_{}_{}.pdf'.format(month,year))
            elif fy_year in [2022,2021,2020,2019,2018]:
                error = True
            
            ## the MPD file naming is wildly inconsistent, so we're trying everything
            
            for file in ["Staffing%20Report_{}_2020{}.pdf".format(month,str(year)[2:]),
                            "Staffing_Report_{}%2020{}.pdf".format(month,str(year)[2:]),
                            "Staffing%20Report_{}%2020{}.pdf".format(month,str(year)[2:]),
                            "Staffing%20Report_Table_{}_2020{}.pdf".format(month,str(year)[2:]),
                            "Staffing%20Report_{}%2020{}.pdf".format(month,str(year)[2:])]:
                try:
                    link = "https://mpdc.dc.gov/sites/default/files/dc/sites/mpdc/page_content/attachments/{}".format(file)
                    filename, headers = opener.retrieve(link, 'Staffing_Report_{}_{}.pdf'.format(month,year))
                    file_downloaded = 1
                    continue
                except Exception as e:
                    error = True

            ## then print out those that failed (ie. have Feb or _updated or something else in the name) or just don't exist
            if file_downloaded == 0:
                failed_files.append('{}_{}'.format(month,year))
        except:                
            print("Error: {}".format(link))
            continue

2020 September
2020 August
2020 July
2020 June
2020 May
2020 April
2020 March
2020 February
2020 January
2020 December
2020 November
2020 October
2019 September
2019 August
2019 July
2019 June
2019 May
2019 April
2019 March
2019 February
2019 January
2019 December
2019 November
2019 October
2018 September
2018 August
2018 July
2018 June
2018 May
2018 April
2018 March
2018 February
2018 January
2018 December
2018 November
2018 October


## Scraping Downloaded MPD PDF Data

In [ ]:
os.chdir('../../data/police/')
files = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('.pdf')]

results = pd.DataFrame()

for file in files:

    print(file)
    
    for page_num in range(0,20):
        try:
            page_num = page_num + 1
            df = tabula.read_pdf(file,pages=page_num,multiple_tables=True)
            df = df[0][~df[0][df[0].columns[0]].isna()]

            result = df[df[df.columns[0]].str.contains("PSA")][[df.columns[0],df.columns[len(df.columns) - 1]]]
            result['file_name'] = file
            result.rename(columns={ result.columns[0]: "PSA" ,
                                    result.columns[1]: "count"}, inplace = True)
            results = pd.concat([results,result])

        except Exception as e:
            break

## Date processing based on file names...

In [188]:
filtered_df = results[results['PSA'].str.contains(r'PSA [0-9][0-9][0-9]', regex=True)]
filtered_df['PSA_cleaned'] = filtered_df['PSA'].str.extract(r'(PSA [0-9][0-9][0-9])', expand=False)

filtered_df['year'] = filtered_df['file_name'].str.extract(r'(20[0-9][0-9])', expand=False)
filtered_df['file_name'] = filtered_df['file_name'].str.lower()
filtered_df['month'] = filtered_df['file_name'].str.extract(r'(january|february|march|april|may|june|july|august|september|october|november|december)', expand=False)

short_to_full_month = {
    'jan': 'january',
    'feb': 'february',
    'mar': 'march',
    'apr': 'april',
    'may': 'may',
    'jun': 'june',
    'jul': 'july',
    'aug': 'august',
    'sep': 'september',
    'oct': 'october',
    'nov': 'november',
    'dec': 'december'
}

filtered_df['file_fix'] = filtered_df['file_name'].replace(short_to_full_month, regex=True)
filtered_df['month_fix'] = filtered_df['file_fix'].str.extract(r'(january|february|march|april|may|june|july|august|september|october|november|december)', expand=False)

filtered_df['month'] = np.where(~filtered_df['month_fix'].isna(), filtered_df['month_fix'], filtered_df['month'])
filtered_df.loc[filtered_df['file_fix'] == 'staffing report_table_10.06.2019.pdf','month'] = 'october'

month_number_mapping = {month: datetime.strptime(month, "%B").month for month in filtered_df['month']}
filtered_df['Numeric_Month'] = filtered_df['month'].map(month_number_mapping)

filtered_df['Numeric_Month'].value_counts(dropna=False)
filtered_df['report_date'] = pd.to_datetime(filtered_df['year'].astype(str) + '-' + filtered_df['Numeric_Month'].astype(str), format='%Y-%m')


/var/folders/y1/yg497znx1yx51vhnw3h2q_v1r7swj4/T/ipykernel_55867/2959591855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['PSA_cleaned'] = filtered_df['PSA'].str.extract(r'(PSA [0-9][0-9][0-9])', expand=False)
/var/folders/y1/yg497znx1yx51vhnw3h2q_v1r7swj4/T/ipykernel_55867/2959591855.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['year'] = filtered_df['file_name'].str.extract(r'(20[0-9][0-9])', expand=False)
/var/folders/y1/yg497znx1yx51vhnw3h2q_v1r7swj4/T/ipykernel_55867

In [197]:
filtered_df[['PSA_cleaned','count','report_date']].sort_values(by=['PSA_cleaned','report_date']).to_csv('staffing.csv',index=False)